# Are you mad enough to sell more clothes?

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

The data set is from a fairly high end clothing chain store in the North East.

You are a data analyst for this store. Your job is to write a report to the pointy-haired boss in which you show how you can increase the store's profit by being targeted about whom to send a catalog in the mail. Yes, you are in direct marketing. You are a quant amongst the "mad men". 

You need to explore and layout in simple terms, what the business needs to spend to increase its profit. In other words, you need a budget, and its your job to figure out how much as well.

We'll guide you through the process. There is much more you can explore, of-course, but this homework will walk you through an entire real world classification and analysis process with a finite amount of work and computer runtime.

You will

1. clean the data, and create some features, learning about how to standardize the data
2. write a classifier on this data, including cross validation, and learn to write a function to encapsulate this process
3. learn how to compare this classifier to baseline classifiers that you better beat using a profit metric rather than an accuracy metric
4. understand and use prediction thresholds
5. understand the use a ROC curve, especially in the situation where probability thresholds are not possible
6. learn to use a profit curve to pick a model, thus directly reflecting the metric of importance
7. learn a bit about feature selection, and why we need to pipeline feature selection and classification together, in an attempt to improve a classifier
8. balance a data set to improve the performance of a SVM classifier
9. implement a kernelized SVM to improve performance further.
10. make a final comparison of classifiers and make a presentation for your boss.

This homework is long because we are walking you through the entire process, start early! There are some parts of it that can be solved by understanding, copying, and slightly modifying code from the lab.  Feel free to do that. Pay attention to any difference in signature in the lab functions to the ones here.

The idea for this homework, and the attendant data set is taken from the book "Data Mining Methods and Models" by [Larose](http://www.dataminingconsultant.com/DMMM.htm). Henceforth we refer to this book as DMMM. There is an analysis of the data set there as well (ch7, the book is available online through our library), which you might be interested in. It is far more detailed than this homework, talking about log-normal data transformations, amongst other things.

(Image credit: www.tabletmag.com)

### Please run all cells before submitting

Get rid of any debugging cells you created, run all the cells you need to run, and then submit. Remember, your notebook is a "product" you are sharing with your fellow data scientists. It must communicate results, code, and text well.

# Table of Contents
* [Are you mad enough to sell more clothes?](#Are-you-mad-enough-to-sell-more-clothes?)
	* [The homework is due Thursday October 22nd at 11.59PM EST.](#The-homework-is-due-Thursday-October-22nd-at-11.59PM-EST.)
		* [Please run all cells before submitting](#Please-run-all-cells-before-submitting)
	* [1. Features and a simple classifier](#1.-Features-and-a-simple-classifier)
		* [Get, check, clean,  the data](#Get,-check,-clean,--the-data)
		* [Feature Engineering](#Feature-Engineering)
			* [Zero important indicators](#Zero-important-indicators)
			* [Combine some features](#Combine-some-features)
		* [Test and Training Sets, and Standardization](#Test-and-Training-Sets,-and-Standardization)
			* [1.1 Explain why this might be a good idea?](#1.1-Explain-why-this-might-be-a-good-idea?)
			* [1.2 Standardize the data](#1.2-Standardize-the-data)
		* [EDA for the data](#EDA-for-the-data)
			* [1.3 EDA for continuous features and indicators](#1.3-EDA-for-continuous-features-and-indicators)
		* [Writing code for a classifier](#Writing-code-for-a-classifier)
			* [1.4 Train a SVM on this data.](#1.4-Train-a-SVM-on-this-data.)
			* [1.5 Write a function to carry out grid-searched cross validation and use it on the SVM](#1.5-Write-a-function-to-carry-out-grid-searched-cross-validation-and-use-it-on-the-SVM)
	* [2. Estimate costs and benefits from assumptions and data](#2.-Estimate-costs-and-benefits-from-assumptions-and-data)
		* [Our data is highly asymmetric](#Our-data-is-highly-asymmetric)
			* [2.1 Compare the accuracy of the SVM to the no-customer-responds baseline](#2.1-Compare-the-accuracy-of-the-SVM-to-the-no-customer-responds-baseline)
		* [Costs and the Confusion Matrix](#Costs-and-the-Confusion-Matrix)
			* [Costs for True Positives, False Negatives, False Positives, and True Negatives](#Costs-for-True-Positives,-False-Negatives,-False-Positives,-and-True-Negatives)
			* [Cost  and Utility Matrix](#Cost--and-Utility-Matrix)
			* [Average Profit Per Person](#Average-Profit-Per-Person)
		* [Establishing Baseline Classifiers via profit](#Establishing-Baseline-Classifiers-via-profit)
			* [Dont Send to Anyone Baseline Classifier](#Dont-Send-to-Anyone-Baseline-Classifier)
			* [2.2 For this baseline classifier question, explain the value we got.](#2.2-For-this-baseline-classifier-question,-explain-the-value-we-got.)
			* [2.3 Send to Everyone Baseline Classifier](#2.3-Send-to-Everyone-Baseline-Classifier)
			* [2.4 Compare the SVM classifier with these baselines](#2.4-Compare-the-SVM-classifier-with-these-baselines)
			* [2.5 Implement logistic regression with Lasso based feature selection](#2.5-Implement-logistic-regression-with-Lasso-based-feature-selection)
		* [Prediction and Re-prediction](#Prediction-and-Re-prediction)
			* [Start with an arbitrary threshold t, and see how we fare at different thresholds for logistic regression](#Start-with-an-arbitrary-threshold-t,-and-see-how-we-fare-at-different-thresholds-for-logistic-regression)
	* [3. Comparing Models with ROC and Profit curves.](#3.-Comparing-Models-with-ROC-and-Profit-curves.)
		* [Thresholds, Comparing Models, and the ROC Curve](#Thresholds,-Comparing-Models,-and-the-ROC-Curve)
			* [3.1 Plot ROC curves](#3.1-Plot-ROC-curves)
		* [Compare classifiers using profit curves](#Compare-classifiers-using-profit-curves)
			* [3.2 Implement a function to calculate average profit per person](#3.2-Implement-a-function-to-calculate-average-profit-per-person)
			* [3.3 Plot profit curves](#3.3-Plot-profit-curves)
	* [4. Final steps: Making decisions, and writing to your boss](#4.-Final-steps:-Making-decisions,-and-writing-to-your-boss)


(This documentation of the fields is taken verbatim from DMMM).

The clothing-store data set contains information about 28,799 customers in the following 51 fields:

- Customer ID: unique, encrypted customer identification `HHKEY`
- Zip code `ZIP_CODE`
- Number of purchase visits `FRE`
- Total net sales `MON`
- Average amount spent per visit `AVRG`
- Amount spent at each of four different franchises (four variables) `AMSPEND`, `PSSPEND`, `CCSPEND`, `AXSPEND`
- Amount spent in the past month, the past three months, and the past six months `OMONSPEND`, `TMONSPEND`, `SMONSPEND`
- Amount spent the same period last year `PREVPD`
- Gross margin percentage `GMP`
- Number of marketing promotions on file `PROMOS`
- Number of days the customer has been on file `DAYS`
- Number of days between purchases `FREDAYS`
- Markdown percentage on customer purchases `MARKDOWN`
- Number of different product classes purchased `CLASSES`
- Number of coupons used by the customer `COUPONS`
- Total number of individual items purchased by the customer `STYLES`
- Number of stores the customer shopped at `STORES`
- Number of promotions mailed in the past year `MAILED`
- Number of promotions responded to in the past year `RESPONDED`
- Promotion response rate for the past year `RESPONSERATE`
- Product uniformity (low score = diverse spending patterns) `HI`
- Lifetime average time between visits `LTFREDAYS`
- Microvision lifestyle cluster type `CLUSTYPE`
- Percent of returns `PERCRET`
- Flag: credit card user `CC_CARD`
- Flag: valid phone number on file `VALPHON`
- Flag: Web shopper `WEB`
- 15 variables providing the percentages spent by the customer on specific classes of clothing, including sweaters, knit tops, knit dresses, blouses, jackets, career pants, casual pants, shirts, dresses, suits, outerwear, jewelry, fashion, legwear, and the collectibles line; (`P*`, `PJACKETS` for example) also a variable showing the brand of choice (encrypted)
- **Target variable**: response to promotion `RESP`...this is our **response** or **y**.


These data are based on a direct mail marketing campaign conducted last year. We want to use this information to develop classification models for this year’s marketing campaign.

## 1. Features and a simple classifier

### Get, check, clean,  the data

In [3]:
df=pd.read_csv("./data/Clothing_Store.csv")
df.head()

In [4]:
df.shape

We'll delete some columns we dont intend to use, and which I couldnt quite figure out what they were from the original data set and documentation.

In [5]:
del df['CLUSTYPE']
del df['HHKEY'], df['ZIP_CODE'], df['REC'], df['PC_CALC20'] 
del df['STORELOY']

In [6]:
df.columns

Make a copy of the dataframe to make transformations to.

In [7]:
dftouse=df.copy()

### Feature Engineering

Feature Engineering is one of the most important "human inputs" that go into machine learning. Machines can run algorithms, but if you feed in garbage, you will get out garbage. The features that are important, or the feature combinations that might be useful in a problem, are inputs that humans can use to help the machine along. Domain knowledge is particularly useful. 

We first list the columns that are percentages:

In [8]:
PERCENT_VARS=[ u'PSWEATERS', u'PKNIT_TOPS', u'PKNIT_DRES', u'PBLOUSES', u'PJACKETS', u'PCAR_PNTS', u'PCAS_PNTS', u'PSHIRTS', 
              u'PDRESSES', u'PSUITS', u'POUTERWEAR', u'PJEWELRY', u'PFASHION', u'PLEGWEAR', u'PCOLLSPND']
len(PERCENT_VARS)

Next, we look for columns where the existence or lack thereof of a zero may be important in a classifier. We used our intuition to make these choices, believing that there is additional information encoded in say, `PERCRET`: if you never returned anything you might not be a budget shopper and thus someone who might have the money to shop quite a bit...

In [9]:
ZERO_IMPORTANT_VARS = [u'PREVPD', u'AMSPEND', u'PSSPEND', u'CCSPEND', u'AXSPEND', u'RESPONDED', u'PERCRET']

We also list the columns with floating-point or integer variables that are amenable to standardization

In [10]:
STANDARDIZABLE = PERCENT_VARS + ZERO_IMPORTANT_VARS + [u'FRE', u'MON',  u'AVRG', u'GMP', u'PROMOS', u'DAYS', u'FREDAYS', u'MARKDOWN', u'CLASSES', u'COUPONS', u'STYLES',  u'MAILED',  u'RESPONSERATE', u'HI', u'LTFREDAY']

Now, an **indicator variable** is one which takes a few, usually 2 values (1/0, True/False) to code the existence or lack thereof of a property or feature. We look for existing indicators:

In [11]:
for v in df.columns:
    l=df[v].unique()
    if len(l) <= 10:
        print v, l

#### Zero important indicators

We encode VALPHON, PERCENT_VARS, and ZERO_IMPORTANT_VARS as indicators. By doing this we are saying: the fact that these features are non-zero carries additional importance as compared to their values.

We maintain a global list INDICATORS in which the names of these columns are stored, prepending an `i_` to each of these variables to denote that they are indicators.

Note that all changes are now being made to the `dftouse` dataframe.

In [12]:
# rename some indicators and make them all 1-0
dftouse['i_VALPHON']=(df.VALPHON=='Y')*1
del dftouse['VALPHON']
dftouse.rename(columns={'WEB':'i_WEB', 'CC_CARD':'i_CC_CARD'}, inplace=True)
INDICATORS=['i_VALPHON','i_WEB','i_CC_CARD']

We then take the `ZERO_IMPORTANT_VARS`, the ones we thought where presence or absence was important, and create indicators from them

In [13]:
for p in ZERO_IMPORTANT_VARS:
    dftouse['i_'+p]=(df[p] > 0.0)*1
    INDICATORS.append('i_'+p)

And then we create indicators for each of the percent variables `PERCENT_VARS` (following Larose's ch7), in the hope that the presence or absence of buying a particular clothing style such as blouses makes a difference...

In [14]:
for p in PERCENT_VARS:
    dftouse['i_'+p]=(df[p] > 0.0)*1
    INDICATORS.append('i_'+p)

#### Combine some features 

We do this to communicate clearly information about recentness and savings

We add two more indicators corresponding to recent spending, and recent use of a savings mechanism.

In [15]:
#create recent usage (1 month and 3 month),  sale-shopper (markdown+coupon)
dftouse["i_RECENT"]=1*((df.OMONSPEND > 0) | (df.TMONSPEND > 0))
dftouse["i_SAVER"]=1*((df.MARKDOWN > 0) | (df.COUPONS > 0))
INDICATORS.append("i_RECENT")
INDICATORS.append("i_SAVER")

Lets see what we now have...

In [16]:
dftouse.head()

In [17]:
dftouse.shape

Clearly we've currently expanded the number of features we have in an attempt to pit in information in the form of indicators which communicate additional distinguishing (in our opinion).

### Test and Training Sets, and Standardization

We standardize test and training sets separately. Specifically, we wish to standardize the non-indicator columns on both the test and training sets, by subtracting out the mean of the training set from the value, and dividing by the standard deviation of the training set. This helps us put all the continuous variables on the same scale.

(There is another reason this might be useful. One optimization which we dont do in this homework but which is useful is to take the log of all positive continuous variables. This makes data look more "normal" which can be useful in some algorithms, and then such standardization can basically be thought of in units of standard deviations of the normal distribution)

#### 1.1 Explain why this might be a good idea?

Why do we do this standardization on the two sets separately?

Hint: what happens to the purity of the training data if we standardize using the entire dataset?

*your answer here*


We'll split the dataset and create a training and test mask.

In [19]:
from sklearn.cross_validation import train_test_split
itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.7)

In [20]:
mask=np.ones(dftouse.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

In [21]:
mask

In [22]:
mask.shape, mask.sum()

#### 1.2 Standardize the data

Use the mask to compute the training and test parts of the dataframe. Use `StandardScaler` from `sklearn.preprocessing` to "fit" the columns in `STANDRARDIZABLE` on the training set. Then use the resultant estimator to transform both the training and the test parts of each of the columns in the dataframe, replacing the old unstandardized values in the `STANDARDIZABLE` columns of `dftouse` by the new standardized ones.

In [23]:
#your code here


We create a list `lcols` of the columns we will use in our classifier. This list should not contain the response `RESP`. How many features do we have?

In [25]:
lcols=list(dftouse.columns)
lcols.remove(u'RESP')
print len(lcols)

### EDA for the data

We create a variable `ccols` which contains all variables not in our indicators list

In [160]:
ccols=[]
for c in lcols:
    if c not in INDICATORS:
        ccols.append(c)
print len(ccols), len(INDICATORS)
ccols

#### 1.3 EDA for continuous features and indicators (SOLVED FOR YOU)

Make a kernel-density estimate plot of each feature in `ccols` and a histogram for each feature in `INDICATORS`. You can use `sns.kdeplot`. In this series of plots color-code the positive and negative responses separately. Do any of the features look promising as separators by themselves?

(HINT: you can create subplots and use plotting functions twice per subplot. There are also other ways.)

In [32]:
dfgb=dftouse.groupby('RESP')
fig, axes = plt.subplots(nrows=11, ncols=4, figsize=(16, 44), 
                         tight_layout=True)
for ax, p in zip(axes.ravel(), ccols):
    for k, v in dfgb[p]:
        sns.kdeplot(v, ax=ax, label=str(k)+":"+v.name)

In [176]:
dfgb=dftouse.groupby('RESP')
fig, axes = plt.subplots(nrows=9, ncols=3, figsize=(16, 44), 
                         tight_layout=True)
for ax, p in zip(axes.ravel(), INDICATORS):
    for k, v in dfgb[p]:
        v.hist(ax=ax, alpha=0.4, label=str(k)+":"+v.name)
        ax.legend()

*your answer here*


### Writing code for a classifier

We will now take this data and write a classifier to predict the response, which is in the `RESP` column of `dftouse`. This response corresponds to asking the question: will a user targeted with our advertisement respond or not?

#### 1.4 Train a SVM on this data.

In [33]:
from sklearn.svm import LinearSVC

We set up a classifier using `LinearSVC`. This algorithm implements a linear SVM. We explicitly pass the standard SVM hinge loss (the default for this classifier in `sklearn` is the squared hinge loss). We choose the linear SVM here as our data size is fairly large, and kernelized SVM training time scales as the square of the number of the samples. Linear SVM, on the other hand, is fast.

In [34]:
clfsvm=LinearSVC(loss="hinge")
Cs=[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
Xmatrix=dftouse[lcols].values
Yresp=dftouse['RESP'].values

In [35]:
Xmatrix_train=Xmatrix[mask]
Xmatrix_test=Xmatrix[~mask]
Yresp_train=Yresp[mask]
Yresp_test=Yresp[~mask]

Use `GridSearchCV` over the parameter grid of regularization coefficients in the `Cs` array to get the best fit classifier using 5-fold cross validation. What value of `C` is the best one to choose? What is the accuracy? (This code will likely take a few minutes to run). Store the best classifier in  variable `best`. Fit the data again with this best classifier, and report on the accuracy of this classifier. HINT: a `GridSearchCV` instance (the variable a `GridSearchCV` object is stored in) has properties such as `best_params_`, `best_score_`, `grid_scores_`, and `best_estimator_`

In [36]:
#your code here


In [37]:
#calculate the accuracy here
#your code here


#### 1.5 Write a function to carry out grid-searched cross validation and use it on the SVM

Lets take the code above and encapsulate it into a function, since we are going to be running many, many classifiers here in this homework. 

You must provide below a function `cv_optimize`, which uses a Grid Search in parameters, and a default of 5-fold cross-validation. It takes as arguments a classifier `clf`, with hyper-parameter dictionary `parameters`, training feature matrix `X`, response `y`, number of folds `n_folds`, and custom scoring function `score_func`. It performs cross-validation using `GridSearchCV` along with a grid-search of hyperparameters. It obtains the best model by the average cross-validation score (allowing for a custom cross-validation score, not just those provided by `sklearn`), and finally returns this best model. (A custom scorer can be provided in the optional `score_func` argument to `cv_optimize`. Its default value should be `None`.)

This `cv_optimize` function is passed to the `do_classify` function below, so you can see how it is to be used before writing it.

In [38]:
"""
Function
--------
cv_optimize

Inputs
------
clf : an instance of a scikit-learn classifier
parameters: a parameter grid dictionary thats passed to GridSearchCV (see above)
X: a samples-features matrix in the scikit-learn style
y: the response vectors of 1s and 0s (+ives and -ives)
n_folds: the number of cross-validation folds (default 5)
score_func: a score function we might want to pass (default python None)
   
Returns
-------
The best estimator from the GridSearchCV, after the GridSearchCV has been used to
fit the model.
     
Notes
-----
see do_classify and the code below for an example of how this is used
"""
#your code here


We use this function to implement a function `do_classify` which provides a general interface to any classifier. This function `do_classify` can take either a `mask` or a `reuse_split` dictionary to specify which is the training set and which is the test set in X and y. In the former case the mask is used to choose which samples to use as training samples and which ones to use as test samples. In the latter case a dictionary is used to directly provide training and test sets. This is useful when the splits already been done and we want to test multiple classifiers on the same test set to compare their performance. An example of the `reuse_split` usage will be seen further down in this notebook. 

We construct the features from `featurenames` columns of the dataframe `indf`, while the column name `targetname` is used with `target1val` as the value that provides a `1` or `+ive` instance. A custom scorer can be provided in the optional `score_func` argument (with default value `None`). `n_folds` is the number of cross-validation folds, set to a default value of 5.

The function prints out the accuracy score on the training and test data, and also the confusion matrix associated with this classifier (more about confusion matrix in the next part of the homework). It returns the best_fit classifier, along with the training and test sets.

Note that the `parameters` dictionary is use to provide a parameter dictionary in the style of `GridSearchCV`, which indeed is used through the `cv_optimize` you provided.

In [39]:
from sklearn.metrics import confusion_matrix
def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    if mask !=None:
        print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

We show how to use this by training the same linear SVM classifier with hinge loss. This classifier will once again be stored in the variable `clfsvm`. This takes a few minutes to run on a circa 2013 Macbook Air. Notice how we use this function carefully, as this will be an example of the usage of this function.

In [40]:
%%time
clfsvm, Xtrain, ytrain, Xtest, ytest = do_classify(LinearSVC(loss="hinge"), {"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'RESP',1, mask=mask)

The results obtained ought to be very similar to the efforts you put in earlier. If not its likely you wrote `cv_optimize` wrong. (Remember that we are using the same mask).

We'll reuse the training and test sets you computed above later in the homework. We do this by putting them into a dictionary `reuse_split`

In [41]:
reuse_split=dict(Xtrain=Xtrain, Xtest=Xtest, ytrain=ytrain, ytest=ytest)

## 2. Estimate costs and benefits from assumptions and data

### Our data is highly asymmetric

First notice that our data set is very highly asymmetric, with positive `RESP`onses only making up 16-17% of the samples.

In [42]:
print "whole data set", dftouse['RESP'].mean()#Highly asymmetric
print "training set", dftouse['RESP'][mask].mean(), "test set", dftouse['RESP'][~mask].mean()

This means that a classifier which predicts that EVERY customer is a negative has an accuracy rate of 83-84%. By this we mean that **a classifier that predicts that no customer will respond to our mailing** has an accuracy of 83-84%!

#### 2.1 Compare the accuracy of the SVM to the no-customer-responds baseline

Based on your comparison, and using accuracy as a metric, does the classifier seem worthwhile pursuing?

*your answer here*


But we havent asked the most important question. Is accuracy really the relevant metric?

### Costs and the Confusion Matrix

Our classifier above had, as one of its printed outputs, a confusion matrix. It looked like this:

In [43]:
ypred=clfsvm.predict(Xtest)
confusion_matrix(ytest, ypred)

The matrix above is of this form:

![hwimages](./images/confusionmatrix.png)


**Important note**: In sklearn, to obtain the confusion matrix in the form above, always have the observed `y` first, i.e.: use as `confusion_matrix(y_true, y_pred)`

In our example, +ives (those with a 1 `RESP`onse) are people who respond to the mailing by going into the store and buying goods. These are also called observed positives (OP). And -ives (those with a 0 `RESP`onse) are those who do not respond to the mailing. These are also called observed Negatives. On our test set, we can print the observed positives and observed negatives respectively:

In [44]:
print "OP=", ytest.sum(), ", ON=",ytest.shape[0] - ytest.sum()

We can make a similar calculation on the predictions of our SVM classifier made on the test set. This gives us the predicted negatives (PN): those customers who we predict will not respond to our mailing; and the predicted positives (PP), the customers who we predict will respond to our mailing by coming into the store to buy stuff.

In [45]:
print "PP=", ypred.sum(), ", PN=",ytest.shape[0] - ypred.sum()

In addition to these four quantities, the confusion matrix gives us more details on proper classifications and mis-classifications from our classifier:

- the samples that are +ive and the classifier predicts as +ive are called True Positives (TP). These are folks we correctly identified as responders,and thus sending them a mailing would result in a sale for us. True Positives are great. We do incur the cost of mailing them, but we like to because they will come into the store to buy.
- the samples that are -ive and the classifier predicts (wrongly) as +ive are called False Positives (FP). False Positives incur us the cost of mailing them as well, but are not very costly. These are people who wouldnt have responded, but we sent them a mailing because our classifier mispredicted them as buyers. Thus, for them, we only incur the cost of preparing the mailing and mailing it to them.
- the samples that are -ive and the classifier predicts as -ive are called True Negatives (TN). These are folks we correctly identified as not-responding, and thus we dont waste any money on sending them a mailing. This is a great classification for us.
- the samples that are +ive and the classifier predicts as -ive are called False Negatives (FN). False negatives are VERY costly: these are folks who would have responded to us had we mailed them, but we didnt target them, leading to huge lost sales per person. Notice that our SVM classifier has tons of False Negatives

It is not enough to simply identify these categories from the confusion matrix. Rather, we want to sit down with our business team and identify the costs associated with each of the 4 classification situations above. Keep in mind that these costs might even change from year to year or even more suddenly: this is why it is important to have marketing and sales people on your data science teams. (See Patil, D. J. Building data science teams. " O'Reilly Media, Inc.", 2011.
 for more details).
 
 Fortunately you have talked to your domain experts and done just that!

#### Costs for True Positives, False Negatives, False Positives, and True Negatives

Lets categorize the costs for each one of these alternatives.

Lets assume the amortized cost of preparing a mailing and mailing it is \$3. Lets assume additionally that the profit margin on a sale is 30% (we are a high end clothing chain).

True Negatives cost us nothing but gain us nothing either.

In [104]:
tnc=0.0 #tnr stands for "true negative cost"

From the average cost of a sale, and the 30% profit assumption, we calculate `tpc`, the cost of a true positive. Note: `tpc` must be negative, since we are talking about costs.

The `tpc` takes into account the cost of mailing to the respondent, and since our mailing works, we subtract out the profit. We use the average of the `AVRG` column, which is the average money spent by a customer on each visit.

In [105]:
prep_and_mail=3
profit_margin=0.3
tpc=prep_and_mail - df.AVRG.mean()*profit_margin 
tpc

The false negative is a lost sale for us! We didnt mail them, and they didnt spend the money. They would have if we mailed them. So we lost a certain profit per such false negative! Thus the false-negative cost, given by `fnc`, is:

In [106]:
fnc = df.AVRG.mean()*profit_margin
fnc

This leaves us with False positives. This is a person who would not have responded but you wasted $3 on. So the false positive cost, (`fpc`) is:

In [107]:
fpc=prep_and_mail
fpc

#### Cost  and Utility Matrix

We then use these costs to write a **risk or cost matrix** in the same form as the confusion matrix above. 

![cost matrix](images/costmatrix.png)

In [109]:
risk_matrix=np.array([[tnc, fpc],[fnc, tpc]])
risk_matrix

Notice that the cost of a false positive is 11 times less than the cost of a false negative. As is often the case in situations in which one class dominates the other, the costs of one kind of misclassification: false negatives are differently expensive than false positives. We saw above that FN are more costly in our case than FP. Similar situations arise in cancer prediction, for example, where a FP only means that you diagnosed a healthy person with cancer, but a FN means that you misdiagnosed a cancer patient as healthy: possibly killing them in the process!

The negative of the cost matrix is called the **utility matrix or profit matrix** `u`. Here we calculate this utility matrix, which we shall use in the next part of the homework.

In [110]:
u = - risk_matrix
u

Ok! Now we can use this profit matrix to calculate the profit that the SVM classifier can land us. 

#### Average Profit Per Person

We can compute the average profit per person using the following formula, which calculates the "expected value" of the per-customer profit (the $P$ below stands for "predicted" and $O$ for observed):



\begin{eqnarray}
Profit &=& u(+P,+O) \times p(+P,+O) \\
       &+& u(+P,-O) \times p(+P,-O) \\
       &+& u(-P,+O) \times p(-P,+O) \\
       &+& u(-P,-O) \times p(-P,-O) 
\end{eqnarray}


which gives


$$ Profit =  \frac{( TP \times -TPC )+ ( FP \times -FPC ) + ( FN \times -FNC ) + ( TN \times -TNC )}{N}$$

where N is the total size of the test set, +P means predicted positive, -O is observed negative, and so on and so forth. The formula above just weighs the profit of a combination of observed and predicted with the out-of-sample probability of the combination occurring. The probabilities are "estimated" by the corresponding confusion matrix on the **test set**, which leads to the second formula. $-TPC$ is just the 'true positive' utility (similar for the others...).

The profit can thus be found by multiplying the utility matrix by the confusion matrix elementwise, and dividing by the sum of the elements in the confusion matrix, or the test set size.

We implement this process of finding the average profit per person in the `average_profit_pp` function below:

In [111]:
def average_profit_pp(y, ypred, u):
    c=confusion_matrix(y,ypred)
    score=np.sum(c*u)/np.sum(c)
    return score

But before we make this calculation for our SVM classifier, we need to first check what profit or cost our baseline classifier which assumes that no customer will respond, incurs.

### Establishing Baseline Classifiers via profit

The simplest classifiers you can think of are the "send to everyone" and "dont send to everyone" classifiers. We explain these below. If we are going to write any more complex classifiers we should at-least outperform these.

#### Dont Send to Anyone Baseline Classifier 

This is the "majority" classifier we talked about earlier. We dont send mailings to anyone because we believe that **no-one will respond**. Thus this classifier predicts everyone to be a 0 or -ive, a non-respondent. Remember, this classifier has a 83-84% accuracy.

We write a confusion matrix `dste` for the "dont send to everyone" model (not the best acronym, I know!), and calculate the average profit per person as `dsteval`. 

In [112]:
testsize = dftouse[~mask].shape[0]
ypred_dste = np.zeros(testsize, dtype="int")
print confusion_matrix(ytest, ypred_dste)
dsteval=average_profit_pp(ytest, ypred_dste, u)
dsteval

#### 2.2 For this baseline classifier question, explain the value we got.

*your answer here*


#### 2.3 Send to Everyone Baseline Classifier

This is the other extreme. In this case we **predict everyone as responders** and send the mailing to everyone. In other words, we predict everyone on the test set to be a 1. Print out both the confusion matrix and `steval`, the average profit per person, for this case. Based on this result, which one of these two classifiers is the one to beat? Why?

In [177]:
#your code here


*your answer here*


#### 2.4 Compare the SVM classifier with these baselines

Using the SVM classifier we calculated, `clfsvm`, calculate the profit we can make

In [114]:
#your code here


If you did this correctly, at this point you might be a bit dejected, and worried about that presentation you have to make to your boss in a few days.

#### 2.5 Implement logistic regression with Lasso based feature selection

In desperation, we run another classifier, a logistic regression with L1 regularization, using the `do_classify` function we defined above. L1 or Lasso regularization automatically does feature selection for us!

Return as the estimator `clflog`, and training and test sets `Xtrain`, `ytrain`, `Xtest`, and `ytest`. Let the regularization hyperparameter `C` range in powers of 10 from 0.001 to 100. Use the `reuse_split` dictionary we calculated earlier. Remember that we want to use "L1" or Lasso regularization: you can do this by passing `penalty="l1"` to the Logistic Regression: `clf=LogisticRegression(penalty="l1")`.

In [115]:
#your code here


Calculate the profit that this classifier gives us:

In [116]:
#your code here


You will find that this logistic regression does a slightly better job, but does little to lift our despondency. What is wrong?

Also what features were removed by the Logistic regression with Lasso?

*your answer here*

### Prediction and Re-prediction

In question 2.1 we asked how the accuracy of our SVM classifier compares to that of the baseline classifiers we tested. However, is this the real question of importance? Our dataset is a very lopsided data set with 84% of samples being negative. In such a case, accuracy is not a very good measure of a classifier.

Then, in question 2.4 and 2.5 we see that the performance of our SVM classifier, even with the "profit" metric instead of the accuracy metric is no big shakes. Indeed, it is outperformed by one of our baseline classifiers. Then we tried the Logistic Regression. That gave us some hope, but even there, the performance turned out to be awful.

What gives?

We were doing something wrong there.

In the case of such asymmetric costs, the `sklearn` API function `predict` is useless, as it assumes a threshold probability of having a +ive sample to be 0.5; that is, if a sample has a greater than 0.5 chance of being a 1, assume it is so. Clearly, when FN are more expensive than FP, you want to lower this threshold: you are ok with falsely classifying -ive examples as +ive. See Lab 5 for how this can be done.

You can think about this very starkly from the perspective of the cancer doctor. Do you really want to be setting a threshold of 0.5 probability to predict if a patient has cancer or not? The false negative problem: ie the chance you predict someone dosent have cancer who has cancer is much higher for such a threshold. You could kill someone by telling them not to get a biopsy. Why not play it safe and assume a much lower threshold: for eg, if the probability of 1(cancer) is greater than 0.05, we'll call it a 1.

Let us do this for our logistic regression example

#### Start with an arbitrary threshold t, and see how we fare at different thresholds for logistic regression

In [117]:
def t_repredict(est,t, xtest):
    probs=est.predict_proba(xtest)
    p0 = probs[:,0]
    p1 = probs[:,1]
    ypred = (p1 > t)*1
    return ypred

We see average profits for multiple thresholds for the logistic regression classifier `clflog`. 

**(a) Average profit per person for t=0.5 (the usual case)**

In [118]:
average_profit_pp(ytest,clflog.predict(Xtest), u)

**(b) Confusion Matrix and average profit per person for t=0.05**

In [119]:
confusion_matrix(ytest,t_repredict(clflog, 0.05, Xtest))

In [120]:
average_profit_pp(ytest, t_repredict(clflog, 0.05, Xtest), u)

**(c) average profit per person for t=0.95**

In [121]:
average_profit_pp(ytest, t_repredict(clflog, 0.95, Xtest), u)

Voila, at a 0.05 threshold we have a nice positive profit! (if you did this right...)

We see that in this situation, where we have asymmetric costs (1:15), we do need to change the threshold at which we make our positive and negative predictions. We need to change the threshold so that we much dislike false nefatives (same in the cancer case). Thus we must accept many more false positives by setting such a low threshold.

For otherwise, we let too many people slip through our hands who would have otherwise shopped at our store. Once we change the threshold, we can make a profit. And indeed, at $t=0.05$, our profit is higher than in the "Send to Everyone" case, which makes doing the classifier worth it! But how do we pick this threshold?

If you read through the lab, you know that we can use the utility or risk matrix to provide a rough threshold to pick for now for our classifier. (You will implement a more powerful method next). The idea is that we want to maximize utility or profit, so for each sample, we simply pick the class which does that. 

There is a caveat for SVM's though: we cannot repredict 1's and 0's directly for  `clfsvm`, as the SVM is whats called a "discriminative" classifier: it directly gives us a decision function, with no probabilistic explanation and no probabilities. (I lie, an SVM can be retrofitted with probabilities: see http://scikit-learn.org/stable/modules/svm.html#scores-probabilities, but these are expensive and not always well callibrated (callibration of probabilities will be covered later in our class)).

What do we do? The SVM does give us a measure of how far we are from the "margin" though, and this is an ordered set of distances, just as the probabilities in a statistical classifier are. This ordering on the distance is just like an ordering on the probabilities: a sample far on the positive side from the line is an almost very definite 1, just like a sample with a 0.99 probability of being a 1 is an almost very definite 1.

To handle the SVM then, and, in the interests of time, we shall skip over fixing thresholds from the utility matrix in this homework, implementing instead the more powerful and generalizable methods of using ROC and profit curves next! Indeed these methods will also help us to deal with the imbalance we have in our data set, with 84% negatives (5 negatives for every positive, roughly).

## 3. Comparing Models with ROC and Profit curves.

### Thresholds, Comparing Models, and the ROC Curve 

ROC curves are a set of classifiers, many of them, each point corresponding to a different threshold (go read Lab 6 for more details). They are useful to compare classifiers to each other and also to baseline models.
(In the standard scenario, where we used the  classifier accuracy, this threshold is implicitly set at 0.5, and we have only one point on the ROC curve.).

The practical way to do this is to order the samples by probability of being positive, or in the case of the SVM, by the `decision_function` or distance from the separating hyperplane. Then consider the sample with the highest score or highest probability of being positive. At first, only this sample is positive. Then, we take the sample with the next highest score, and call it positive. As we go down the list, we go down a threshold in score or probability. 

Now, for each such situation: only 1 positive, now 2 positive,....you can imagine a different classifier with a different confusion matrix. It will have its own false positives, three positives, etc. Its actually the same original classifier, but with a different threshold each time.

As we keep going down the list, decreasing the threshold, more and more samples become positive, and at first, the true positives rise faster than the false positives. Once past a certain point, false positives increase faster than true positives. Now, if you want a balanced classifier, you look at this turn-around point...the northwest corner, so to speak. But if you want a classifier which penalizes false positives and false negatives differently, the point you want is different.

To make a ROC curve you plot the True Positive Rate, 

$$TPR=\frac{TP}{OP}$$

against the False Positive Rate,

$$FPR=\frac{FP}{ON}$$

as you go through this process of going down the list of samples. ROC curves are useful because they calculate one classifier per threshold and show you where you are in TPR/FPR space without making any assumptions about the utility matrix or which threshold is appropriate.


A rote reading of the ROC curve (go to the "northwest" corner) is a bad idea: you must fold in the curve with any assumptions you are making about the utilities. In our case we have both an asymmetric data set, and asymmetric risk, so the north west corner may not be the right spot. Still, on the whole, a curve with a greater AUC (area under curve), or further away from the line of randomness, will give us a rough idea of what might be a better classifier.

We provide example code `make_roc` for you to calculate a ROC curve. Pass `proba=False` to `make_roc` for non-probabilistic classifiers: notice how we use `decision_function` here instead of `predict_proba`.

This function uses `roc_curve` from sklearn: note that this function carries out the process we described and gives us back a triplet of `fpr`, `tpr`, and `thresholds`. We also calculate the `auc`, and plot the roc curve with thresholds, skipping every `labe` number of thresholds to make labels.

In [122]:
from sklearn.metrics import roc_curve, auc
def make_roc(name, clf, ytest, xtest, ax=None, labe=5, proba=True, skip=0):
    initial=False
    if not ax:
        ax=plt.gca()
        initial=True
    if proba:#for stuff like logistic regression
        fpr, tpr, thresholds=roc_curve(ytest, clf.predict_proba(xtest)[:,1])
    else:#for stuff like SVM
        fpr, tpr, thresholds=roc_curve(ytest, clf.decision_function(xtest))
    roc_auc = auc(fpr, tpr)
    if skip:
        l=fpr.shape[0]
        ax.plot(fpr[0:l:skip], tpr[0:l:skip], '.-', alpha=0.3, label='ROC curve for %s (area = %0.2f)' % (name, roc_auc))
    else:
        ax.plot(fpr, tpr, '.-', alpha=0.3, label='ROC curve for %s (area = %0.2f)' % (name, roc_auc))
    label_kwargs = {}
    label_kwargs['bbox'] = dict(
        boxstyle='round,pad=0.3', alpha=0.2,
    )
    if labe!=None:
        for k in xrange(0, fpr.shape[0],labe):
            #from https://gist.github.com/podshumok/c1d1c9394335d86255b8
            threshold = str(np.round(thresholds[k], 2))
            ax.annotate(threshold, (fpr[k], tpr[k]), **label_kwargs)
    if initial:
        ax.plot([0, 1], [0, 1], 'k--')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title('ROC')
    ax.legend(loc="lower right")
    return ax

Here is an example of the usage of this function. Play with it to see how to label things

In [123]:
#notice for the first plot we do not pass an axes object. Subsequent calls to
#make_roc can use the returned axes object ax.
with sns.hls_palette(8, l=.3, s=.8):
    ax=make_roc("logistic-with-lasso",clflog, ytest, Xtest, labe=200, skip=50)

#### 3.1 Plot ROC curves

Plot ROC curves for the logistic regression classifier `clflog`, and linear SVM classifier `clfsvm` below. Label the curve for the former classifier `logistic-with-lasso`, and the latter classifier, `svm-all-features` . Choose colors using `with sns.color_palette("palletename")` that make the curves easily separable. (see https://stanford.edu/~mwaskom/software/seaborn/tutorial/color_palettes.html)

In [124]:
#your code here


### Compare classifiers using profit curves

The proof is always in the pudding. So far we have seen the ROC curve which implements one classifier per threshold to pick an appropriate model. But why not just plot the profit on a ROC like curve to see which classifier maximizes profit? 

Just like in a ROC curve, we go down the sorted (by score or probability) list of samples. We one-by-one add an additional sample to our positive samples, noting down the attendant classifier's TPR and FPR and threshold. In addition to what we do for the ROC curve, we now also note down the percentage of our list of samples predicted as positive. Remember we start from the mostest positive, where the percentage labelled as positive would be minuscule, like 0.1 or so and the threshold like a 0.99 in probability or so. As we decrease the threshold, the percentage predicted to be positive clearly increases until everything is predicted positive at a threshold of 0. What we now do is, at each such additional sample/threshold (given to us by the `roc_curve` function from `sklearn`), we calculate the expected profit per person and plot it against the percentage predicted positive by that threshold to produce a profit curve. Thus, small percentages correspond to samples most likely to be positive: a percentage of 8% means the top 8% of our samples ranked by likelihood of being positive.

We provide code to plot a profit curve below, to which we must provide two critical functions:

- code to calculate expected profit given the TPR and FPR from a classifier (this is different than our `average_profit_pp` above as we now want this in terms of TPR and FPR.
- code to calculate the percentage of samples classified positive.given the TPR and FPR of a classifier.


We have implemented the function `percentage(tpr, fpr, priorp, priorn)` to calculate the percentage of samples classified positive at a given tpr/fpr (ie a classifier for a given threshold).

The variable `priorp` is the fraction of actual positive samples in the test set (OP), and `priorn` is the percentage of actual -ive (0) samples in the test set (ON).

In [130]:
def percentage(tpr, fpr, priorp, priorn):
    perc = tpr*priorp + fpr*priorn
    return perc

#### 3.2 Implement a function to calculate average profit per person

Implement a function `av_profit(tpr, fpr, util, priorp, priorn)` to calculate average profit per person given the utility matrix, the FPR rate, the TPR rate, and class balance.

You will need to look at the diagram of the confusion matrix, and the formula for profit, to derive the formula used in the function you must implement. Present your derivation below the code.


In [131]:
"""
Function
--------
av_profit

Inputs
------
tpr: true positive rate
fpr: false positive rate
util: utility matrix for this problem
priorp: the probability of observed +ives (OP) on our test set
priorn: the probability of observed +ives (ON) on our test set

   
Returns
-------
The average profit per person at this (fpr, tpr) point in this ROC space.
     
Notes
-----
see make_profit below for an example of how this is used
"""
#your code here


*your answer here*


Given the definition of `percentage` and `av_profit`, we can now write code to make a profit curve.

In [132]:
def make_profit(name, clf, ytest, xtest, util, ax=None, threshold=False, labe=200, proba=True):
    initial=False
    if not ax:
        ax=plt.gca()
        initial=True
    if proba:
        fpr, tpr, thresholds=roc_curve(ytest, clf.predict_proba(xtest)[:,1])
    else:
        fpr, tpr, thresholds=roc_curve(ytest, clf.decision_function(xtest))
    priorp=np.mean(ytest)
    priorn=1. - priorp
    ben=[]
    percs=[]
    for i,t in enumerate(thresholds):
        perc=percentage(tpr[i], fpr[i], priorp, priorn)
        ev = av_profit(tpr[i], fpr[i], util, priorp, priorn)
        ben.append(ev)
        percs.append(perc*100)
    ax.plot(percs, ben, '-', alpha=0.3, markersize=5, label='utlity curve for %s' % name)
    if threshold:
        label_kwargs = {}
        label_kwargs['bbox'] = dict(
        boxstyle='round,pad=0.3', alpha=0.2,
        )
        for k in xrange(0, fpr.shape[0],labe):
            #from https://gist.github.com/podshumok/c1d1c9394335d86255b8
            threshold = str(np.round(thresholds[k], 2))
            ax.annotate(threshold, (percs[k], ben[k]), **label_kwargs)
    ax.legend(loc="lower right")
    return ax

#### 3.3 Plot profit curves

Plot the profit curves using the function above for the logistic regression (`clflog`) and linear svm (`clfsvm`). Keep the same labels as the ROC curves. Also annotate on the graph (use matplotlib's `ax.annotate` function) the points on the profit curve corresponding to baseline models send-to-everyone annotated as STE (from `steval`) and dont-send-to-everyone annotated as DSTE (from `dsteval`). 

Comment on (a) why you only start making a profit at a certain percentage, and (b) what is the region on interest on the graph that you might want to communicate with the company managers in terms of maximizing your profit over the baseline (Hint: you might need to draw a horizontal line on this graph to communicate this region) (c) Which classifier makes us a greater profit?

In [133]:
#your code here


*your answer here*


#### 4 Choose the best performing classifier and use it to write a 5-6 slide presentation for your boss

Suppose that based on this targeted mailing from the previous year, you want to decide how many customers of your 100,000 person customer base you ought to target. Your boss has proposed a budget of $45,000

Include the information asked for below. Remember your audience is a pointy haired boss:

![pointy](http://assets.amuniversal.com/12e53860415d01300e80001dd8b71c47)

1. What the problem is?
2. What are the baselines..
3. Is a budget of dollars 45,000 adequate? ( Remember the amortized cost of preparing the mailer and mailing it out per person is dollars 3 ). 
4. How profit changes with budget? Where is it maximized?
5. Your recommendations for the boss.

Create the slides using Google Presentations, including any additional annotated figures you made in your repository's hw3 branch. Copy the url of your presentation into the cell below, and include any additional computation at the bottom ofter the url.

*your answer here*
